<a href="https://colab.research.google.com/github/eddy123457/AI_Upscaler_CNN/blob/main/AI_upscalerCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#train model here
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import ssl
# cancels out the SSL requirement
ssl._create_default_https_context = ssl._create_unverified_context

#data set is from kereas includes uptp 60,000 images
(x_train_lowres, _), (x_test_lowres, _) = cifar10.load_data()
#normalize pixels [0,1] with low res
x_train_lowres = x_train_lowres.astype('float32') / 255.0
x_test_lowres = x_test_lowres.astype('float32') / 255.0
#resize the low res images
x_train_highres = tf.image.resize(x_train_lowres, (64, 64)).numpy()
x_test_highres = tf.image.resize(x_test_lowres, (64, 64)).numpy()

def upscalingV2():
    #we use seq model because it allows more layers to be added
    model = models.Sequential()
    #first layer 64 filters and a filter has a (3,3) kernal
    #Rectified Linear unit is used
    #padding is the same so we always have the same spatial dimensions
    #input_shape allows for anysized imaged to be used on model
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(None, None, 3)))
    model.add(layers.BatchNormalization())
    #same as before but we add 128 filteres
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())

    #heres where we do the actual upscaling by factor of two
    #takes way to long to increase this
    model.add(layers.UpSampling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    #we use this since we normalized first
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))

    return model

upscalingmodelV2 = upscalingV2()
#adam optimizer with learning rate of .001 ans mse is used to calulate loss on training epoch
upscalingmodelV2.compile(optimizer=Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())

upscalingmodelV2.summary()

# Save the model during training just in case my computer crashes again
checkpoint = ModelCheckpoint('eds_upscaler_model.h5', save_best_only=True)

# Train the model 10 epochs and a batch size of 32 images consisting of low and high res imgs
history = upscalingmodelV2.fit(x_train_lowres, x_train_highres, epochs=10, batch_size=32,
                              validation_data=(x_test_lowres, x_test_highres),
                              callbacks=[checkpoint])

# Display model statistics
print("Training History:")
print(history.history)

# Evaluate the model on the test set
test_loss = upscalingmodelV2.evaluate(x_test_lowres, x_test_highres)
print(f"\nTest Loss: {test_loss}")

# Save the final model
upscalingmodelV2.save('eds_upscaler_modelV2.h5')


In [ ]:

#import model here
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model


imagepath = "img"
image = cv2.imread(imagepath)

# Resize the Image to what we want
size = (1920, 1080)
resizedImage = cv2.resize(image, size)

# make sure its a colored image
if len(resizedImage.shape) == 2:
    resizedImage = cv2.cvtColor(resizedImage, cv2.COLOR_GRAY2RGB)

# normalize the img
normalizedImage = resizedImage.astype('float32') / 255.0

# show side by side original and normalized image
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

plt.subplot(1, 2, 2)
plt.title("Normalized Image")
plt.imshow(normalizedImage)
plt.show()

preprocessedImage=normalizedImage
preprocessedImage = cv2.cvtColor(preprocessedImage, cv2.COLOR_BGR2RGB)

# Load the my pre trained upscaling model
upscalingModelPath = "//eds_upscaler_modelV2.h5"
upscalingModel = load_model(upscalingModelPath)

# Expand dimensions to match the model's expected input shape
inputImage = tf.expand_dims(preprocessedImage, axis=0)

# Use the model to generate the high-resolution image
generated_image = upscalingModel.predict(inputImage)

# Display the Original and Generated Images
# Expand dimensions to match the model's expected input shape
inputImage = tf.expand_dims(preprocessedImage, axis=0)

# create a new picture
generatedimage = upscalingModel.predict(inputImage)

#show result
plt.subplot(1, 2, 1)
plt.title("Preprocessed Image")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

plt.subplot(1, 2, 2)
plt.title("Generated Image")
plt.imshow(generatedimage[0])  # Access the first  element in the batch
plt.show()